# Análise de texto de fontes desestruturadas e Web

## Exercícios da Aula 09

Este notebook servirá para relembrarmos **requests** e **BeautifulSoup**, que serão utilizadas para baixar notícias do site da **IstoÉ Dinheiro**. Em seguida, iremos extrair informações para construir um Pandas DataFrame de títulos e descrições para prática de **RegEx**.

## Importando as bibliotecas necessárias

In [ ]:
# para nos comunicarmos com a Web
import requests
import urllib

# para extrair informações de páginas HTML
import bs4
from bs4 import BeautifulSoup

# Para criar um Data Frame
import pandas as pd

# Para expressões regulares
import re

# Para PDFs
import PyPDF2 as pp

# Recursos do sistema
import os

# Aleatoriedade
import random

# Para sleep
import time

Caso necessário, utilize `pip` e faça instalação das bibliotecas:

In [ ]:
# !pip install --upgrade pip
# !pip install --upgrade PyPDF2

## Definindo cabeçalho User-Agent

In [ ]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
]

Vamos sortear aleatoriamente um *user agent*:

In [ ]:
u_agent = random.choice(user_agents)
headers = ({"User-Agent": u_agent})

## Definindo em qual página buscar

Vamos definir qual seção iremos utilizar para baixar as notícias.

In [ ]:
secao = "economia"
url = f"https://istoedinheiro.com.br/categoria/{secao}/"

## Utilizando *requests* para baixar a página de notícias

Com o uso da biblioteca **requests**, podemos obter o **HTML** da página da IstoÉ.

In [ ]:
resposta = requests.get(url=url, headers=headers)

resposta.encoding = "utf-8"

print(resposta.text)

## Extraindo informações relevantes com *BeautifulSoup*

Perceba que o HTML inclui uma grande quantidade de tags, o que dificulta identificar informações relevantes de forma direta. Com o auxílio da biblioteca **BeautifulSoup** podemos extrair facilmente as informações que desejamos.

In [ ]:
soup = BeautifulSoup(resposta.text, "html.parser")

Agora, vamos obter uma lista com todos os trechos HTML que contém uma notícia. Para identificar as tags corretas, é preciso ir até a página Web que desejamos extrair informações, dar botão direito e ir em **inspecionar elemento**, navegando pelo HTML até identificar as tags necessárias

Ex: https://istoedinheiro.com.br/categoria/economia/

In [ ]:
# Noticias com a thumb image
lista_tag_individual = soup.find_all("article", attrs={"name": "individualNew"})

# Notícias sem imagem ao lado do título
lista_tag_a_text = soup.find_all("article", attrs={"name": "ArticleWithText"})

# Concatena as duas listas
lista_tag_noticia = lista_tag_individual + lista_tag_a_text

# Todas as notícias encontradas
lista_tag_noticia

Agora, podemos passar por cada uma das chamadas de notícias, extraindo informações de interesse, como o título, descrição e data.

In [ ]:
lista_titulo = []
lista_desc = []
lista_data = []

for i in range(0, len(lista_tag_noticia)):
    
    tag_noticia = lista_tag_noticia[i]

    # Encontra o título
    titulo = tag_noticia.find("h1").text
    titulo = titulo.replace("\n", "") #limpa os ENTERS
    lista_titulo.append(titulo)

    #  Encontra a descrição
    descricao = tag_noticia.find("p").text
    lista_desc.append(descricao)
    
    # Encontra a data e hora
    data_hora = tag_noticia.find("span").text
    lista_data.append(data_hora)

## Criando um DataFrame

As informações que consideramos foram extraídas na repetição **for** e armazenadas em listas. Podemos utilizar estas listas para construir um Pandas DataFrame:

In [ ]:
df = pd.DataFrame({"Secao": secao,
                   "Titulo": lista_titulo,
                   "Descrição": lista_desc,
                   "Data": lista_data
                  })
df

## Fazer o scraping de várias páginas

Ao navegar pelo site, percebemos que a **URL** das páginas segue o seguinte padrão: https://www.istoedinheiro.com.br/categoria/economia/page/2/

Vamos construir algumas funções auxiliares e fazer a extração dos dados das notícias de várias páginas:

In [ ]:
def get_headers():
    u_agent = random.choice(user_agents)
    headers = ({"User-Agent": u_agent})
    return headers

def sleepy_code():
    sleep_time = random.uniform(1, 2) 
    time.sleep(sleep_time)

def download_page(secao = "economia"):
    sleepy_code()
    url = f"https://istoedinheiro.com.br/categoria/{secao}/"
    resposta = requests.get(url = url, headers=get_headers())
    resposta.encoding = "utf-8"
    return resposta.text

def get_data(secao = "economia"):
    html = download_page(secao=secao) 
    soup = BeautifulSoup(html, "html.parser")
    
    # Noticias com a thumb image
    lista_tag_individual = soup.find_all("article", attrs={"name": "individualNew"})
    # Notícias sem imagem ao lado do título
    lista_tag_a_text = soup.find_all("article", attrs={"name": "ArticleWithText"})
    # Concatena as duas listas
    lista_tag_noticia = lista_tag_individual + lista_tag_a_text

    lista_titulo = []
    lista_desc = []
    lista_data = []

    for tag_noticia in lista_tag_noticia:

        titulo = tag_noticia.find("h1").text
        titulo = titulo.replace("\n", "") #limpa os ENTERS
        lista_titulo.append(titulo)

        descricao = tag_noticia.find("p").text
        lista_desc.append(descricao)
        
        data_hora = tag_noticia.find("span").text
        lista_data.append(data_hora)

    return lista_titulo, lista_desc, lista_data

def get_dataframe(secao = "economia"):
    lista_titulo, lista_desc, lista_data = get_data(secao=secao)
    df = pd.DataFrame({"Secao": secao,
                       "Titulo": lista_titulo,
                       "Descrição": lista_desc,
                       "Data": lista_data
                      })
    return df

def get_news(secoes):
    dfs = []
    for secao in secoes:
        dfs.append(get_dataframe(secao=secao))
    return pd.concat(dfs, axis=0)        

Vamos extrair dados de algumas seções:

In [ ]:
secoes = ["economia", "negocios", "mercado-digital", "carreira", "financas", "giro"]
random.shuffle(secoes)

secoes

In [ ]:
df = get_news(secoes)

E visualizar algumas notícias

In [ ]:
print(f"Temos {len(df)} noticias!")

Vamos conferir, aleatoriamente, algumas das notícias baixadas:

In [ ]:
df.sample(10)

### Salvando o DataFrame em CSV

É interessante armazenar o Dataframe em CSV para que ele possa ser análisado em algum momento posterior. Você poderia, por exemplo, extrair as notícias todos os dias de uma semana e analisar somente após ter todos estes dados.

In [ ]:
df.to_csv("noticias_180424.csv", index=False)

## Relembrando - Extração de textos de PDFs

Vamos relembrar o que vimos na aula de extração de textos de PDFs e juntar com Expressões Regulares para procurarmos por padrões interessantes.

Primeiro, vamos fazer o download de uma página qualquer do diário oficial.

In [ ]:
pdf_url="http://diariooficial.imprensaoficial.com.br/doflash/prototipo/2023/Abril/19/exec1/pdf/pg_0001.pdf"

response = urllib.request.urlopen(pdf_url)

In [ ]:
response.status

Vamos salvar a resposta obtida em um arquivo PDF chamado `"pg_0001.pdf"`

In [ ]:
arq = open("pg_0001.pdf", "wb")
arq.write(response.read())
arq.close()

Então podemos utilizar a biblioteca `PyPDF2` para extrair os textos. Aqui, pense que nosso objetivo será apenas identificar **CPF**, **CNPJ**, datas, projetos de leis, etc. mencionados, não importanto tanto que o texto esteja em ordem.

In [ ]:
pp_reader = pp.PdfReader(open("pg_0001.pdf", "rb"))

texto = pp_reader.pages[0].extract_text()
print(texto)

### Capturando CPFs

Vamos procurar por todos os CPFs mencionados nesta página:

In [ ]:
re.findall(r"\d{3}\.\d{3}\.\d{3}-\d{2}", texto)

E procurar por todas as datas

In [ ]:
re.findall(r"\b\d{1,2}/\d{1,2}/\d{2,4}\b", texto)

**Pergunta**: Qual a utilidade do `\b` no regex acima?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.

</div>

**Pergunta**: Como listar todos os RGs?

In [ ]:
# Seu código aqui!

<a href="#" title="re.findall(r'RG\s\d+', texto)">Pare o mouse em cima deste link para ver a resposta!</a>

# Exercícios

Considere a base de notícias recém extraída para os exercícios.

**Exercício 1)** Utilize RegEx para criar uma nova coluna no DataFrame. Esta nova coluna deve indicar se o título da notícia tem faz ou não menção ao **governo**.

Aqui, você vai ter que pensar em um RegEx que busque por termos que façam sentido e generalizem a noção de **"governo"**.

In [ ]:
# Seu código aqui!

**Exercício 2)** Repita o exercício anterior, buscando por termos que façam menção à **bolsas de valores**.

In [ ]:
# Seu código aqui!

**Exercício 3)** Repita o exercício anterior, buscando por termos que façam menção à **bolsas de valores** ou ao **governo** na **descrição** das notícias.

In [ ]:
# Seu código aqui!

**Exercício 4)** Utilizando as variáveis novas criadas com RegEx:


**a)** Conte quantas notícias fazem menção ao Governo. Conte para o título e também descrição.

In [ ]:
# Seu código aqui!

**b)** Gere um gráfico de barras das frequências absolutas (contagem)

In [ ]:
# Seu código aqui!

**c)** E se quisermos a frequência relativa (porcentagem)

In [ ]:
# Seu código aqui!

**Exercício 5)** Calcule a frequência de menções à Pandemia por seção (política, economia, finanças). Obs: Considere a descrição da notícia. Gere um gráfico de barras dos resultados.

In [ ]:
# Seu código aqui!

**Exercício 6)** Crie um código python que consiga fazer o download de várias páginas do diário oficial.

Salve em arquivos nomeados no padrão `"pg_0001.pdf"`, `"pg_0002.pdf"`, ... , `"pg_000n.pdf"`

In [ ]:
# Seu código aqui!

**Exercício 7)** Crie um código python que leia `n` arquivos **PDF**s de uma pasta e extraia seus textos utilizando a biblioteca vista no exemplo da aula.

Aqui, cada **PDF** é uma página do diário oficial. Suponha que os arquivos estão nomeados no padrão `'pg_0001.pdf'`, `'pg_0002.pdf'`, ... , `'pg_000n.pdf'`.

Retorne uma lista onde cada item da lista é uma string contendo o texto da página em questão. Exemplo:

```python
[
    'texto da página 01 do diário',
    'texto da página 02 do diário',
    'texto da página 03 do diário',
    'texto da página 04 do diário',
]
```

In [ ]:
# Seu código aqui!

**Exercício 8)** Crie um código python que recebe a lista do exercício anterior. Exemplo:

```python
[
    'texto da página 01 do diário',
    'texto da página 02 do diário',
    'texto da página 03 do diário',
    'texto da página 04 do diário',
]
```

Você deve procurar todos os **CPF**s ou **CNPJ**s contidos no diário. Indique a página onde o mesmo foi encontrado. Exemplo de resposta:
```python
[
    ['123.456.789-00', 0],
    ['87.340.538/0001-23', 0]
    ['555.666.777-00', 1],
    ['30.375.316/0001-29', 3],
    ['30.375.316/0001-29', 3],
]
```

In [ ]:
# Seu código aqui!

**Exercício 9)** Um empresário deseja saber quais páginas do diário oficial fazem menção a determinado assunto.

Faça um programa em python que recebe os textos das páginas do diário oficial:


```python
[
    'texto da página 01 do diário',
    'texto da página 02 do diário',
    'texto da página 03 do diário',
    'texto da página 04 do diário',
]
```

Crie um padrão de expressão regular para busca por termos/palavras (simule um cenário) e retorne uma lista das páginas que fazem menção ao padrão. Exemplo de resposta:


```python
[0, 2, 3]
```

In [ ]:
# Seu código aqui!